In [62]:
import numpy as np
import math
import random

In [338]:
def read_data(test):
    path = "job"+str(test)+".assign"
    f = open(path, "r")
    txt = f.readlines()
    mtrx = []
    matrix = []
    for i,line in enumerate(txt):
        mtrx.append(line.split(' '))
        matrix.append([])
        for j,num in enumerate(mtrx[i]):
            if num != '':
                matrix[i].append(num)
    if test == 1:
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                matrix[i][j] = int(matrix[i][j])
    
    elif test == 2:
        for i in range(len(matrix)):
            matrix[i].pop(-1)
            for j in range(len(matrix[i])):
                matrix[i][j] = int(matrix[i][j])
    return matrix

In [339]:
class Ant:
    def __init__(self):
        self.path = []
        self.cost = 0
    
    def reset(self):
        self.path = []
        self.cost = 0
    

In [340]:
class Colony:
    def __init__(self, ant_num, matrix , base_formun, T_SA, SA_rate, evaporate_rate, beta, p_local, gamma_Qlearning, alpha, q_0):
        self.n = len(matrix)
        self.matrix = matrix
        self.ant_num = ant_num
        self.T = T_SA
        self.evaporate_rate = evaporate_rate
        self.beta = beta
        self.alpha = alpha
        self.p = p_local
        self.gamma_Qlearning = gamma_Qlearning
        self.base_formun = base_formun
        self.q_0 = q_0
        
        self.formun = [[base_formun for i in range(self.n)] for j in range(self.n)]

        self.ants = []
        for i in range(ant_num):
            ant = Ant()
            self.ants.append(ant)
        
        
    def training(self,itteration=100):
        for itter in range(itteration):
            print(itter,end=': ')
            self.spread_ants()
            self.formun = [[self.base_formun for i in range(self.n)] for j in range(self.n)]

            for step in range(self.n - 1):
                for ant in self.ants:
                    self.move(ant)
                    self.local_update_formun(ant.path[-2],ant.path[-1])
                    
            self.global_update_formun()
            for ant in self.ants:
                ant.reset()
            

    def spread_ants(self):
        for i,ant in enumerate(self.ants):
            if i < self.n:
                ant.path.append(i)
                ant.cost += self.matrix[0][i]
            else:
                state = random.choice(list(range(self.n)))
                ant.path.append(state)
                ant.cost += self.matrix[0][state]



    def local_update_formun(self,i,j):
#         delta = 0
#         for z in range(self.n):
#             if z != j and z != i:
#                 temp = self.gamma_Qlearning * self.formun[j][z]
#                 if temp > delta:
#                     delta = temp

        self.formun[i][j] = ((1-self.p)* self.formun[i][j]) + (self.p*self.base_formun)

    def find_best_path(self):
        optima_path = []
        best_cost = float('inf')
        
        for ant in self.ants:
            if ant.cost < best_cost:
                best_cost = ant.cost
                optima_path = ant.path

        print(best_cost)
#         print(optima_path)
        return optima_path, best_cost

    def global_update_formun(self):
        path, cost = self.find_best_path()
        for i in range((self.n)-1):
            self.formun[i][i+1] += (self.alpha*(cost**-1))
        
        self.formun[path[-1]][path[0]] += (self.alpha*(cost**-1))
        
        for i in range(self.n):
            for j in range(self.n):
                self.formun[i][j] = (1-self.alpha)*self.formun[i][j] 

    def select_next_node(self,ant):
        q = random.random()
        if q < self.q_0:
            maxx = 0
            selected = None
            for node in range(self.n):
                if not node in ant.path:
                    rule = self.formun[ant.path[-1]][node] * ((1/self.matrix[len(ant.path)][node])**self.beta)
                    if rule > maxx:
                        maxx = rule
                        selected = node
            return selected

        else:
            prob = [0 for i in range(self.n)]
            summ = 0
            for node in range(self.n):
                if not node in ant.path:
                    summ += self.formun[ant.path[-1]][node] * ((1/self.matrix[len(ant.path)][node])**self.beta)

            for node in range(self.n):
                if not node in ant.path:
                    prob[node] = (self.formun[ant.path[-1]][node] * ((1/self.matrix[len(ant.path)][node])**self.beta))/summ
                    a = random.choices(list(range(self.n)), prob, k=1)
            return a[0]

    def move(self,ant):
        next_node = self.select_next_node(ant)
        ant.cost += self.matrix[len(ant.path)][next_node]
        ant.path.append(next_node)

                

In [344]:
test = read_data(1)
colony = Colony(ant_num = 10, 
                matrix = test ,
                base_formun = 1/10, 
                T_SA = 1, 
                SA_rate = 0.99, 
                evaporate_rate = 0.9, 
                beta = 2, 
                p_local = 0.1, 
                gamma_Qlearning = 0.7, 
                alpha = 0.1,
                q_0 = 0.5)
colony.training(1000)

0: 56
1: 38
2: 56
3: 31
4: 31
5: 56
6: 31
7: 26
8: 26
9: 56
10: 26
11: 26
12: 56
13: 26
14: 26
15: 26
16: 31
17: 26
18: 56
19: 31
20: 31
21: 38
22: 56
23: 26
24: 31
25: 26
26: 31
27: 31
28: 31
29: 26
30: 26
31: 56
32: 31
33: 26
34: 61
35: 56
36: 26
37: 26
38: 56
39: 31
40: 26
41: 31
42: 26
43: 43
44: 56
45: 26
46: 26
47: 56
48: 38
49: 26
50: 26
51: 56
52: 56
53: 56
54: 43
55: 56
56: 44
57: 31
58: 31
59: 26
60: 26
61: 56
62: 31
63: 43
64: 31
65: 56
66: 56
67: 31
68: 26
69: 26
70: 43
71: 31
72: 26
73: 56
74: 56
75: 31
76: 31
77: 31
78: 26
79: 31
80: 26
81: 26
82: 56
83: 31
84: 56
85: 31
86: 56
87: 56
88: 26
89: 38
90: 31
91: 26
92: 26
93: 56
94: 31
95: 43
96: 43
97: 31
98: 31
99: 56
100: 26
101: 43
102: 26
103: 56
104: 56
105: 26
106: 56
107: 56
108: 56
109: 31
110: 31
111: 31
112: 56
113: 31
114: 31
115: 31
116: 56
117: 38
118: 31
119: 56
120: 38
121: 26
122: 56
123: 31
124: 56
125: 31
126: 26
127: 56
128: 31
129: 26
130: 44
131: 26
132: 31
133: 31
134: 31
135: 31
136: 26
137: 31
138: 2

In [323]:
np.asarray(matrix)

array([['5', '10', '3', '4\n'],
       ['8', '7', '49', '19\n'],
       ['6', '10', '9', '28\n'],
       ['6', '10', '39', '289\n']], dtype='<U4')